In [3]:

import functions as hdi_functions
import pandas as pd
import numpy as np
import sql_functions
import sqlalchemy
from init_database import init_data_base

init_data_base()

data_base_name='sustainable_development'
con = sql_functions.get_connection()

sql = 'select * from inputs;'
inputs = pd.read_sql_query(sql=sql, con=con)


def compute_outputs(threshold_max_MF, threshold_min_MF, threshold_max_CDE, threshold_min_CDE, threshold_max_GNI, inputs: pd.DataFrame):
    columns = ['entity_name', 'entity_id', 'sustainable_development_index',
               'hdi', 'factor_planetary_pressure', 'life_expectancy_index', 'education_index', 'income_index', 'carbon_dioxide_emission_index', 'material_footprint_index']
    result_list = []
    for index, row in inputs.iterrows():
        mfi = hdi_functions.compute_material_footprint_index(
            observed_value_MF=row['material_footprint'], threshold_max_MF=threshold_max_MF, threshold_min_MF=threshold_min_MF)

        cdei = hdi_functions.compute_carbon_dioxide_emissions_index(
            observed_value_CDE=row['carbon_dioxide_emission'], threshold_max_CDE=threshold_max_CDE, threshold_min_CDE=threshold_min_CDE)

        fpp = hdi_functions.compute_factor_planetary_preassures(
            cdei=cdei, mfi=mfi)

        ei = hdi_functions.compute_education_index(
            row['mean_years_schooling'], row['expected_years_schooling'])

        ii = hdi_functions.compute_income_index(
            row['gross_national_income'], threshold_max_GNI=threshold_max_GNI)

        lei = hdi_functions.compute_life_expectancy_index(
            row['life_expectancy'])

        hdi = hdi_functions.compute_human_development_index(
            lei=lei, ei=ei, ii=ii)

        sustainable_development = hdi_functions.compute_sustainable_development(
            hdi=hdi, fpp=fpp)

        new_row = [row['entity_name'], row['entity_id'], sustainable_development,  hdi,
                   fpp, lei, ei, ii, cdei, mfi]
        result_list.append(new_row)

    np_arr = np.array(result_list)
    result = pd.DataFrame(np_arr, columns=columns)
    return result


onu_df = compute_outputs(107.42, 0, 68.72, 0, 75000, inputs)

onu_countries_data = onu_df[onu_df["entity_id"] == '1']
onu_countries_df = pd.DataFrame(onu_countries_data)
onu_countries_df['ranking'] = onu_countries_df['sustainable_development_index'].rank(
    ascending=False, method='max')
onu_countries_df.to_sql(con=con, name='outputs_countries_{tag}'.format(
    tag='onu'), if_exists='replace')


onu_others_data = onu_df[onu_df["entity_id"] != '1']
onu_others_df = pd.DataFrame(onu_others_data)
onu_others_df['ranking'] = onu_others_df['sustainable_development_index'].rank(
    ascending=False, method='max')

onu_others_df.to_sql(con=con, name='outputs_others_{tag}'.format(
    tag='onu'), if_exists='replace')


revised_df = compute_outputs(50, 0, 38, 0, 75000, inputs)
revised_countries_data = revised_df[revised_df["entity_id"] == '1']
revised_countries_df = pd.DataFrame(revised_countries_data)
revised_countries_df['ranking'] = revised_countries_df['sustainable_development_index'].rank(
    ascending=False, method='max')

outputs_data_types = {
    'entity_id': sqlalchemy.Integer,
    'sustainable_development_index': sqlalchemy.Float,
    'hdi': sqlalchemy.Float,
    'factor_planetary_pressure': sqlalchemy.Float,
    'life_expectancy_index': sqlalchemy.Float,
    'education_index': sqlalchemy.Float,
    'carbon_dioxide_emission_index': sqlalchemy.Float,
    'material_footprint_index': sqlalchemy.Float,
    'income_index': sqlalchemy.Float,
}
revised_countries_df.to_sql(con=con, name='outputs_countries_{tag}'.format(tag='revised'), if_exists='replace',
                            dtype=outputs_data_types)


revised_others_data = revised_df[revised_df["entity_id"] != '1']
revised_others_df = pd.DataFrame(revised_others_data)
revised_others_df['ranking'] = revised_others_df['sustainable_development_index'].rank(
    ascending=False, method='max')

revised_others_df.to_sql(con=con, name='outputs_others_{tag}'.format(
    tag='revised'), if_exists='replace', dtype=outputs_data_types)
